In [2]:
import altair as alt
import pandas as pd

In [41]:
# Define custom theme to be applied to all plots
def theme():
    return {
        "config": {
            "title": {
                "dy": 1,
                "fontSize": 22,
                "fontWeight": 400,
                "align": "center",
                "anchor": "middle",
                "subtitleColor": "grey",
                "subtitleFontSize": 22
            },
            "view": {
                "fill": "#FCFCFC",
            },
            "header": {
                "titleFontSize": 23,
                "labelFontSize": 23,
                "labelFontWeight": 400,
            },
            "axis": {
                "titleFontSize": 23,
                "titleFontWeight": 400,
                "labelFontSize": 20,
                "labelFontWeight": 400,
                "labelLimit": 1000,
                "domainWidth": 1.5,
                "domainColor": "black",
                "tickColor": "black",
                "domain": False
            },
            "axisX": {
                "domain": True
            },
            "legend": {
                "titleFontSize": 23,
                "titleFontWeight": 400,
                "labelFontSize": 23,
                "labelLimit": 1000,
                "strokeColor": '#F4F6F7',
                "padding": 15
            }
        }
    }

alt.themes.register("theme", theme)
alt.themes.enable("theme")

ThemeRegistry.enable('theme')

In [42]:
"""
The name of the folder you created under `../data/`
"""
TIME_STAMP_FOLDER_NAME = '08-01-2024'

In [118]:
df = pd.read_csv(f'../data/{TIME_STAMP_FOLDER_NAME}/manual/Manual Test Results.csv')

df['TASK'] = 'T' + df.TASK_ID.astype(str) + '. ' + df.TASK_SHORT_DESC

df.SECONDS = df.SECONDS.apply(lambda x: 300 if x > 300 else x)

df['MINUTES'] = df.SECONDS
df.MINUTES /= 60

df['IS_SUCCESS'] = df.SUCCESS.apply(lambda x: 'Success' if x == 'O' else 'Fail')
df.SUCCESS = df.SUCCESS.apply(lambda x: 'Succeeded in 5 mins' if x == 'O' else 'Failed in 5 mins')

df.BASELINE = 4

df.FRUSTRATION = 8 - df.FRUSTRATION

df

TASKS_SORTED = df.groupby('TASK').agg({'SECONDS': 'mean'}).reset_index().sort_values(by='SECONDS', ascending=False).TASK.tolist()
TASKS_SORTED

df.head(3)

,DATA_PORTAL,TASK_ID,TASK_SHORT_DESC,TASK_LONG_DESC,SUCCESS,SECONDS,CONFIDENCE,SATISFACTION,FRUSTRATION,TASK,MINUTES,IS_SUCCESS
0,HuBMAP,1,Sign in,Sign in to the HuBMAP data portal,Failed in 5 mins,300,4,4,5,T1. Sign in,5.000000,Fail
1,HuBMAP,2,Find publications,Find a list of publications that used HuBMAP d...,Succeeded in 5 mins,106,5,5,4,T2. Find publications,1.766667,Success
2,HuBMAP,3,Filter datasets,Find kidney datasets for donors over the age o...,Failed in 5 mins,300,3,3,5,T3. Filter datasets,5.000000,Fail


In [121]:
plot_result = alt.Chart(df).mark_bar().encode(
    alt.X('mean(MINUTES):Q', title='Time taken (minutes)').scale(domain=[0, 5], clamp=True).axis(format='.0f', tickCount=5),
    alt.Y('TASK:N', title='Tasks', sort=TASKS_SORTED).axis(titleX=-360, zindex=1),
    # alt.Color('SUCCESS:N', title='Task completion').scale(range=['#D55D00', '#56B4E9']).legend(orient='bottom'),
    # alt.Row('DATA_PORTAL:N', title=None)
).properties(
    title={
        'text': 'Task Completion',
        # 'subtitle': 'Data as of Jul 24, 2024',
        "dy": -10
    },
    width=400,
    height=300
)
plot_result#.resolve_scale(x='independent')

alt.Chart(...)

In [185]:
# suc = alt.Chart(df).mark_point(filled=True, size=400).encode(
#     # alt.X('SUCCESS:Q', title=''),
#     alt.Y('TASK:N', title=None, sort=TASKS_SORTED).axis(None),
#     alt.Opacity('IS_SUCCESS:N', legend=None).scale(domain=['Success', 'Fail'], range=[1, 0]),
#     # alt.Color('SUCCESS:N', title='Task completion').scale(range=['#D55D00', '#56B4E9']).legend(orient='bottom'),
#     alt.Column('DATA_PORTAL:N', title=None, spacing=1).header(labelAngle=-90, labelAnchor='end', labelAlign='right')
# ).properties(
#     title={
#         'text': 'Task Success',
#         # 'subtitle': 'Data as of Jul 24, 2024',
#         "dy": -10
#     },
#     width=40,
#     height=300
# )
# suc

In [186]:
piedf = df.groupby(['TASK', 'SUCCESS']).agg({'IS_SUCCESS': 'count'}).reset_index()
piedf.IS_SUCCESS /= 3

pie = alt.Chart(piedf).mark_arc().encode(
    alt.Theta('IS_SUCCESS', title='').stack(True),
    alt.Row('TASK:N', title='Tasks', sort=TASKS_SORTED,spacing=2).header(None),#.axis(titleX=-360, zindex=1),
    # alt.Opacity('IS_SUCCESS:N', legend=None).scale(domain=['Success', 'Fail'], range=[1, 0]),
    alt.Color('SUCCESS:N', title='Task completion').scale(domain=['Succeeded in 5 mins', 'Failed in 5 mins'], range=['#4C78A8', 'lightgrey']).legend(None),
    # alt.Column('DATA_PORTAL:N', title=None, spacing=1).header(labelAngle=-90, labelAnchor='end', labelAlign='right')
).properties(
    title={
        'text': 'Succeded',
        # 'subtitle': 'Data as of Jul 24, 2024',
        "dy": -10
    },
    width=40,
    height=29
)
# pie#.resolve_scale(x='independent')
# piedf
pie

alt.Chart(...)

In [187]:
COLS = ['CONFIDENCE', 'SATISFACTION', 'FRUSTRATION']

df['baseline'] = 4

plot_likert = None
for c in COLS:
    _plot = alt.Chart(df).mark_point(size=150, opacity=1, filled=True, color='black').encode(
        alt.X(f'mean({c}):Q', title=[f'{c.title()}']).scale(domain=[1, 7] if c != 'FRUSTRATION' else [7, 1]).axis(tickCount=5), # , '(1=negative, 7=positive)'
        alt.Y('TASK:N', title='Tasks', sort=TASKS_SORTED).axis(None),
        # alt.Color('CONFIDENCE:N', title=None, legend=None).scale(domain=[1, 2, 3, 4, 5, 6, 7], range=['#56B4E9', '#56B4E9', '#56B4E9', 'grey', '#D55D00', '#D55D00', '#D55D00']),
    ).properties(
        title={
            'text': f'{c.title()} Level',
            # 'subtitle': 'Data as of Jul 24, 2024',
            "dy": -10
        },
        width=300,
        height=300
    )

    err = _plot.mark_errorbar().encode(
        x=alt.X(f'{c}:Q', title=[f'{c.title()}']).scale(zero=False),
        color=alt.value('grey')
    )
    
    rule = alt.Chart(df).mark_rule(color='black', strokeDash=[3, 3]).encode(
        alt.X('baseline:Q').scale(domain=[1, 7] if c != 'FRUSTRATION' else [7, 1])
    )
    
    _plot = rule + err + _plot

    # _plot = _plot.facet(row=alt.Row('DATA_PORTAL:N', title=None, header=None))

    if plot_likert == None:
        plot_likert = _plot
    else:
        plot_likert |= _plot

plot_likert = plot_likert.resolve_scale(y='shared')

In [189]:
plot = alt.hconcat(plot_result, pie, plot_likert, spacing=0).resolve_scale(y='shared').properties(
    # title={
    #     'text': 'User Testing Results of HuBMAP Data Portal',
    #     'subtitle': 'Data as of Jul 24, 2024',
    #     "dy": -10,
    #     'fontSize': 30
    # },
)
# plot.save(f'../data/{TIME_STAMP_FOLDER_NAME}/figures/manual-test-results.png', scale_factor=8)

"""
Save for the manuscript figures and website plots
"""
NAME = 'manual-test-results'
plot.save(f'../data/{TIME_STAMP_FOLDER_NAME}/figures/{NAME}.png', scale_factor=8)
plot.save(f'../data/{TIME_STAMP_FOLDER_NAME}/website/{NAME}.json')

plot

alt.HConcatChart(...)